
<div dir="rtl">
<h3>نمونه‌های ربات تلگرام</h3>
</div>

<br />

<a href="https://www.youtube.com/watch?v=30c1pkpkeSU&list=PLY_JJbHwd6kftjlcbtChIKZfEPEfFRGcY&index=5&ab_channel=TheInternationalJournalists%27Network">
<img src="https://dotnet-talk.com/images/icfj/3_3.jpg" width="50%" alt="نمونه ربات های تلگرم - ویدیو" />
</a>

<br />

<div dir="rtl">
این درس، بر اساس درس قبلی بنا شده است و به شما کمک می‌کند تا احساس کنید که چه کارکردهای مفیدی با یک ربات ممکن است انجام شود و کد کلیدی مورد نیاز برای تولید آن کارکرد را فراهم می‌کند. ربات‌های تلگرام می‌توانند ابزارهای بسیار ارزشمندی برای خبرنگاران باشند. در اینجا، چند نمونه ارائه می‌دهیم. برای برخی از این نمونه‌ها، کد کلیدی مورد نیاز برای راه‌اندازی ربات را ارائه می‌دهیم - دستوراتی که کاربر می‌تواند برای تعامل با ربات استفاده کند.
</div>

<div dir="rtl">
۱. جمع‌آوری داده: ربات‌ها می‌توانند برای جمع‌آوری داده‌ها از کاربران، مانند پاسخ‌های نظرسنجی یا گزارشات شاهدان مورد استفاده قرار گیرند.
</div>

In [ ]:
def message(update: Update, context: CallbackContext) -> None:
    # 'context.args' is a list of arguments passed after the command.
    # We join them into a string 'user_said'.
    user_said = ' '.join(context.args)

    # The bot sends a message back to the user saying what the user said.
    update.message.reply_text(f"You said: {user_said}")
    
    # Open the file 'user_messages.txt' in append mode ('a'). If the file does not exist, it gets created.
    # 'with' keyword is used for context management. It automatically closes the file after we're done.
    with open('user_messages.txt', 'a') as f:
        # Write the user's message at the end of the file. '\n' is a newline character that separates messages.
        f.write(f"\n{user_said}")


<div dir="rtl">
۲. ترویج محتوا: اگر یک خبرنگار یک مقاله یا پست وب جدید دارد، یک ربات می‌تواند به‌طور خودکار آن را با دنبال‌کنندگان خود به اشتراک بگذارد.
</div>

In [ ]:
def latest(update: Update, context: CallbackContext) -> None:
    # Open the file 'latest_info.txt' in read mode ('r').
    # If the file does not exist, Python will throw an error.
    with open('latest_info.txt', 'r') as f:
        # Read the entire content of the file into the string 'latest_info'.
        latest_info = f.read()

    # The bot sends the content of 'latest_info' as a message to the user.
    update.message.reply_text(latest_info)

<div dir="rtl">
۳. ربات‌های پرسش و پاسخ: این ربات‌ها می‌توانند برنامه‌ریزی شوند تا به سؤالات متداول درباره یک خبر یا یک موضوع پاسخ دهند و خبرنگار را از نیاز به پاسخ به هر سؤال به‌طور جداگانه رها کنند.
    <br />
۴. ساخت جامعه: ربات می‌تواند برای کمک به تشکیل و مدیریت یک جامعه دیجیتال از شهروندان علاقه‌مند استفاده شود، اطلاعاتی درباره رویدادهای محلی، هماهنگی در تظاهرات صلح‌آمیز و یا سازماندهی کمک به نیازمندان به اشتراک بگذارد.
    <br />
۵. نظرسنجی و بررسی‌ها: ربات می‌تواند نظرسنجی‌ها و بررسی‌ها را اجرا کند و به جمع‌آوری نظرات عمومی درباره مسائل مهم محلی یا ملی کمک کند.
</div>

In [ ]:
import telegram
from telegram.ext import CommandHandler, MessageHandler, Updater, Application, filters
import asyncio
import nest_asyncio
import requests
import logging

nest_asyncio.apply()

# General constants
TELEGRAM_TOKEN = "TELEGRAM_TOKEN"
ACU_APIKEY = "ACU_APIKEY"
Language = "fa-IR"
WAITING = 0

logging.basicConfig(filename='c:\\logs\\bot.log', encoding='utf-8', level=logging.INFO)

async def start(update, context):
    message = "به ربات هواشناسی خوش آمدید \r\n"
    message += "برای دریافت وضعیت هوای شهر مورد نظر /weather را بنویسید"
    await context.bot.send_message(chat_id=update.effective_chat.id, text=message)

async def weather(update, context):
    global WAITING
    WAITING = 1
    message = "نام شهر را وارد کنید:\r\n"
    print(update.message)
    await context.bot.send_message(chat_id=update.effective_chat.id, text=message)

async def handle_message(update, context):
    global WAITING
    message = update.message

    if WAITING != True:
        await context.bot.send_message(chat_id=message.chat_id, text="ورودی خطا! دوباره تلاش کنید.")
        await start(update, context)
        return

    await context.bot.send_message(chat_id=message.chat_id, text="شما %s را وارد کرده اید" % (message.text))

    try:
        get_location_url = "http://dataservice.accuweather.com/locations/v1/search?apikey=%s&q=%s&language=%s" % (ACU_APIKEY, message.text, Language)
        logging.info(get_location_url)

        location_response = requests.get(get_location_url)
        
        logging.info(location_response.json()[0])
        type = location_response.json()[0]["Type"]
        region = location_response.json()[0]["Region"]["LocalizedName"]
        country = location_response.json()[0]["Country"]["LocalizedName"]
        timezone = location_response.json()[0]["TimeZone"]["Code"]
        key = location_response.json()[0]["Key"]
        
        get_weather_url = "http://dataservice.accuweather.com/forecasts/v1/daily/1day/%s?apikey=%s&language=%s&details=true&metric=true" % (key, ACU_APIKEY, Language)
        logging.info(get_weather_url)

        weather_response = requests.get(get_weather_url)
        logging.info(weather_response.json())

        await context.bot.send_message(chat_id=message.chat_id, text="منطقه وارد شده یک %s می باشد." % (type))
        await context.bot.send_message(chat_id=message.chat_id, text="این شهر در کشور %s قرار دارد." % (region))
        await context.bot.send_message(chat_id=message.chat_id, text="این شهر در منطقه %s قرار دارد." % (country))
        await context.bot.send_message(chat_id=message.chat_id, text="این شهر در منطقه زمانی %s قرار دارد." % (timezone))

        effective_date = weather_response.json()["Headline"]["EffectiveDate"]
        text = weather_response.json()["Headline"]["Text"]
        min_temperatur = weather_response.json()["DailyForecasts"][0]["Temperature"]["Minimum"]["Value"]
        max_temperatur = weather_response.json()["DailyForecasts"][0]["Temperature"]["Maximum"]["Value"]

        await context.bot.send_message(chat_id=message.chat_id, text="تاریخ امروز %s" % (effective_date))
        await context.bot.send_message(chat_id=message.chat_id, text="وضعیت هوا %s" % (text))
        await context.bot.send_message(chat_id=message.chat_id, text="کمینه دما %s و بیشینه دما %s می باشد." % (min_temperatur, max_temperatur))

    except Exception as e:
        logging.error(str(e))
        await context.bot.send_message(chat_id=message.chat_id, text="شهر وارد شده در دیتابیس AcuWeather وجود ندارد.")
        await start(update, context)

    WAITING = False

async def bot_start():
    application = Application.builder().token(TELEGRAM_TOKEN).build()

    application.add_handler(CommandHandler("start", start))
    application.add_handler(CommandHandler("weather", weather))
    application.add_handler(MessageHandler(filters.ALL, handle_message))
    application.run_polling()

await bot_start()

Message(channel_chat_created=False, chat=Chat(first_name='Mat', id=5161600953, type=<ChatType.PRIVATE>), date=datetime.datetime(2023, 7, 10, 22, 26, 4, tzinfo=<UTC>), delete_chat_photo=False, entities=(MessageEntity(length=8, offset=0, type=<MessageEntityType.BOT_COMMAND>),), from_user=User(first_name='Mat', id=5161600953, is_bot=False, language_code='en'), group_chat_created=False, message_id=533, supergroup_chat_created=False, text='/weather')
Message(channel_chat_created=False, chat=Chat(first_name='Mat', id=5161600953, type=<ChatType.PRIVATE>), date=datetime.datetime(2023, 7, 10, 22, 26, 38, tzinfo=<UTC>), delete_chat_photo=False, entities=(MessageEntity(length=8, offset=0, type=<MessageEntityType.BOT_COMMAND>),), from_user=User(first_name='Mat', id=5161600953, is_bot=False, language_code='en'), group_chat_created=False, message_id=544, supergroup_chat_created=False, text='/weather')
Message(channel_chat_created=False, chat=Chat(first_name='Mat', id=5161600953, type=<ChatType.PRIVA

<div dir="rtl">
<p>همین است! توجه کنید که همه این ایده‌ها می‌توانند با رباتی که در دفترچه یادداشت قبلی ساختیم، یکپارچه شوند.</p>
</div>